# Abstract type

We define an abstract type to represent a point in a two-dimensional space.  This type will be used as a parent type for concrete types.

In [1]:
abstract type AbstractPoint end

This type will be subtyped to create types that have certain properties, in this case $x$ and $y$ coordinates.

It is also considered good practice to provide accessor function to deal with attributes, rather than accessing them directly.

In [2]:
get_x_coord(p::AbstractPoint) = p.x
set_x_coord(p::AbstractPoint, new_x::Real) = begin p.x = new_x; end
get_y_coord(p::AbstractPoint) = p.y
set_y_coord(p::AbstractPoint, new_y::Real) = begin p.y = new_y; end

set_y_coord (generic function with 1 method)

These accessor methods will work on all subtypes of `AbstractPoint` as long as as the structure that implements it has fields `x` and `y`.

We define two methods, the first one to compute the Euclidean distance between two points, the other to move a point by a given distance in the $x$ and $y$ direction.

In [3]:
function distance(p1::AbstractPoint, p2::AbstractPoint)::AbstractFloat
    return sqrt((get_x_coord(p1) - get_x_coord(p2))^2 + (get_y_coord(p1) - get_y_coord(p2))^2)
end

distance (generic function with 1 method)

Note that this method is defined on the abstract type `Point2D`, not on any concrete type.  This means that it can be used on a structure of any bype subtyped of `Point2D`.

In [4]:
function move(p::AbstractPoint, delta_x::Real, delta_y::Real)
    set_x_coord(p, get_x_coord(p) + delta_x)
    set_y_coord(p, get_y_coord(p) + delta_y)
    return nothing
end

move (generic function with 1 method)

# Simple point

We define a type that represents a point in a two dimensional space using real-valued coordinates.  The `Point` structure has two fields, the $x$ and $y$ coordinate of the point.  Note that the type of the coordinates is abstract, i.e., it can be any of `Float16`, `Float32` or `Float64` since it is declared `Real`.  The concrete type `Point` is a subtype of the abstract tyee `AbstractPoint`.

In [5]:
mutable struct Point <: AbstractPoint
    x::Real
    y::Real
end

We try out the methods `destance` and `move` on two points.

In [6]:
p1 = Point(3.1, 4.5)
p2 = Point(2.3, -1.9);

In [7]:
distance(p1, p2)

6.44980619863884

In [8]:
move(p1, 0.3, 0.2)

In [9]:
p1

Point(3.4, 4.7)

# Particle

In Julia, a concrete type can not be subtyped, so the only way to extend a type is by composition.  A particle has an $x$ and $y$ coordinate and we can compute the distance between two particles.  These methods have been defined for the abstract type `AbstractPoint`, and to ensure that they work on `Particle` as they do on `Point`, we can symply ensure that `Particle` has an `x` and `y` field.

A particle has mass and a velocity ($x$ and $y$ components).  It is also a subtype of the abstract type `AbstractPoint`.

In [10]:
mutable struct Particle <: AbstractPoint
    x::Real
    y::Real
    mass::Real
    v_x::Real
    v_y::Real
end

Since `Patricle` is a subtype of `Point2D`, we can simply use the `distance` and `move` method we already defined.

In [11]:
p1 = Particle(1.2, 2.3, 1.5, -9.0, -7.0)
p2 = Particle(3.1, 4.2, 1.3, 8.0, 2.0);

In [12]:
distance(p1, p2)

2.687005768508881

The `move` method for particles reuses the `move` method defined for `Point`.

In [13]:
function move(p::Particle, delta_t::Real)
    move(p, p.v_x*delta_t, p.v_y*delta_t)
    return nothing
end

move (generic function with 2 methods)

In [14]:
move(p1, 0.1)

In [15]:
p1

Particle(0.29999999999999993, 1.5999999999999996, 1.5, -9.0, -7.0)

Note that we are using multiple dispatch, i.e., `move` has two methods, one for `Point2D` and the other for `Patricle`.

# Composition

Although it may seem overkill to define `get_x_coord`, `set_x_coord` and friends, it helps when you would have to use composition to extend a type.

In [16]:
mutable struct ColoredPoint <: AbstractPoint
    coords::Point
    color::String
    
    ColoredPoint(x::Real, y::Real, color::String) = new(Point(x, y), color)
end

The representation of the $x$ and $y$ coordinates is fundamentally different from that of the `Point` and `Particle` type, we can still ensure that the methods `distance` and `move` work if we reimplement the accessor methods for the coordinates.  We could of course just reimplement them, but the `Lazy` module defines a handy macro to *forward* the call to an element of `ColoredPoint`.

In [17]:
using Lazy

In [18]:
@forward ColoredPoint.coords get_x_coord
@forward ColoredPoint.coords set_x_coord
@forward ColoredPoint.coords get_y_coord
@forward ColoredPoint.coords set_y_coord

Thanks to the inner constructor, a `ColoredPoint` can also be constructed by simply specifying the $x$ and $y$ coordinates, rather than by explicitly constructing a Point as for `p1`.

In [19]:
p1 = ColoredPoint(1.2, -0.5, "red")
p2 = ColoredPoint(-1.3, 2.5, "blue")

ColoredPoint(Point(-1.3, 2.5), "blue")

In [20]:
get_x_coord(p1)

1.2

In [21]:
distance(p1, p2)

3.905124837953327

# Traits

The cleaner way to do things is by defining traits.  A trait is an abstract type, the name ending by convention either in `Style` or `Trait`.  The trait `ColoredStyle` will be used that some types have *color*.

There are also at least two concrete types subtyped from `ColoredStyle`, one to indicate that a type has color, the other that it hasn't.  Note that some traits can have more than two options.

In [22]:
abstract type ColoredStyle end
struct IsColored <: ColoredStyle end
struct IsNotColored <: ColoredStyle end;

Traits can be assigned to types by defining methods that take types are arguments and using multiple dispatch.  The default for any type is that it is not colored, so the first function will return an `IsNotColored` struct object.  The second method will be invoked on all types that subtype `ColoredPoint` (which has a color) and return an `IsColored` structure as a result.

These functions can be used to implement the behavior of methods depending on the traits their arguments have.

In [23]:
ColoredStyle(::Type) = IsNotColored()
ColoredStyle(::Type{<:ColoredPoint}) = IsColored()

ColoredStyle

We now define a method `get_color` that is supposed to return the color of a structure that has the trait `ColoredStyle`, and that generates an error if its argument has not.

The first method for `get_color` takes a single argument, the object we want to get the color of.  The type of that object is treated as a variable `T`, and the function will call one of two other methods, based on whether `T` has the `ColoredStyle` trait or not.  For the former, the method `ColoredSthle` will return an instance of `IsColored`, for the latter an instance of `IsNotColored`.  Two additional methods for `get_color` deal with these respective cases.

In [24]:
get_color(x::T) where {T} = get_color(ColoredStyle(T), x)
get_color(::IsColored, x) = x.color
get_color(::IsNotColored, x) = error("$(typeof(x)) is not colored");

In [25]:
point = Point(-1.2, 2.4)
colored_point = ColoredPoint(1.2, -7.3, "blue")

ColoredPoint(Point(1.2, -7.3), "blue")

When we try to get the color of a non-colored object, we get the appropriate error message, while if the object is colored, `get_color` will return its value.

In [38]:
try
    get_color(point)
catch err
    println(err.msg)
end

Point is not colored


In [27]:
get_color(colored_point)

"blue"

Similarly, "pointiness" could also be defined as a trait.

In [28]:
abstract type PointStyle end
struct IsPoint <: PointStyle end
struct IsNotPoint <: PointStyle end

In [29]:
PointStyle(::Type) = IsNotPoint()
PointStyle(::Type{<:AbstractPoint}) = IsPoint();

We can now define methods `get_x`, `set_x`, `get_y` and `set_y`.

In [30]:
get_x(p::T) where {T} = get_x(PointStyle(T), p)
get_x(::IsPoint, p) = get_x_coord(p)
get_x(::IsNotPoint, p) = error("$(typeof(p)) is not a pointy type")

set_x(p::T, value::Real) where {T} = set_x(PointStyle(T), p, value)
set_x(::IsPoint, p, value) = set_x_coord(p, value)
set_x(::IsNotPoint, p, value) = error("$(typeof(p)) is not a pointy type")

get_y(p::T) where {T} = get_y(PointStyle(T), p)
get_y(::IsPoint, p) = get_y_coord(p)
get_y(::IsNotPoint, p) = error("$(typeof(p)) is not a pointy type")

set_y(p::T, value::Real) where {T} = set_y(PointStyle(T), p, value)
set_y(::IsPoint, p, value) = set_y_coord(p, value)
set_y(::IsNotPoint, p, value) = error("$(typeof(p)) is not a pointy type")s

set_y (generic function with 3 methods)

In [31]:
get_x(point)

-1.2

In [32]:
get_x(colored_point)

1.2

All this now amounts to `Point`, `Particle` and `ColoredPoint` having the `PointStyle` since all three are subtypes of `AbstractPoint`, and `ColoredPoint` having the `ColoredStyle` trait in addition to that.

The nice thing about traits is that you don't need a class hierarchy such as `AbstractPoint`.  For instance, if we have a type `Paint` that has a color, we can simply give that the `ColoredStyle` trait.

In [33]:
struct Paint
    color::String
end

Assigning the trait to the `Paint` type is as simple as adding another method for the `ColoredStyle` function, restricting the type of the argument to subtypes of `Paint`, and returning `IsColored`.

In [34]:
ColoredStyle(::Type{<:Paint}) = IsColored();

In [35]:
blue_paint = Paint("red")

Paint("red")

In [36]:
get_color(blue_paint)

"red"

Note that naming a paint the is red `blue_paint` is not really recommended.

Of course, `Paint` has no coordinates.

In [42]:
try
    get_x(blue_paint)
catch err
    println(err.msg)
end

Paint is not a pointy type
